In [48]:
import io, json, codecs, os, fix

In [50]:
def redif2jsonld(jsonld_src,journal_name,issn,file_name):
    global articles; articles=[]
    global article; article={}
    global authors; authors=[]
    global author; author=''
    global e
    for i in content:
        pair=i.split(':')
        j=i.replace(pair[0]+':','').strip()
        key=pair[0].lower()
        if 'template-type' in key:
            if len(article)>0:
                if len(author)>0: authors.append(author); author=''
                article['e:author']=authors
                authors=[]
                if fix.validate(article)==True: articles.append(article)
                else: e=e+1
            article={}
            article['e:author']=[]
            article['e:keyword']=[]
            article['e:journal']=journal_name
            article['e:issn']=issn
            article['e:title']=''
            article['e:year']=0
        elif key=='author-name':
            if len(author)>0: authors.append(author)
            author=''
            if ',' in j: j=' '.join(reversed([part.strip() for part in j.split(',')]))
            author=j
        elif key=='title':article['e:title']=j
        elif key=='year': 
            try: article['e:year']=int(j) 
            except: pass
        elif key=='pages':article['e:pages']=j
        elif key=='volume':article['e:volume']=j
        elif key=='issue':article['e:issue']=j
        elif key=='file-url':
            article['e:url']=j
            article['e:doi']=j.replace('http://dx.doi.org/','').replace('http://hdl.handle.net/','')
        elif key=='doi':article['e:doi']=j
        elif key=='abstract':article['e:abstract']=j
        elif key=='keywords':
            for keyword in j.replace(';',',').split(','):
                if keyword.strip()!='': article['e:keyword'].append(fix.fix_keyword(keyword))

    if len(article)>0:
        if len(author)>0: authors.append(author); author=''
        article['e:author']=authors; authors=[]
        articles.append(article)   
        
    jd=unicode(json.dumps(articles, ensure_ascii=False))
    jsonld_file=jsonld_src+issn+'_'+file_name.replace('.redif','').replace('.rdf','')+'.jsonld'
    #if not os.path.exists(jsonld_file):
    with io.open(jsonld_file, 'w', encoding="utf-8") as f:
        f.write(jd)
    return (jsonld_file)

In [53]:
redif_folder='redif/'
jsonld_src='jsonld_src/'

    
with open('../OpenScience/subject_journals.json') as data_file:    
    subject_journals = json.load(data_file)

journals=[]
for subject in subject_journals:
    journals=journals+(subject['journals'])
print len(journals)
    
for journal in journals:
    source=journal['jname']
    #if source not in  ['ormnsc','ormsom']: continue
    journal_name=journal['journal']
    data=journal['data']
    issn=journal['issn']
    if issn !='':
        for type_pub in data:   
            file_type=type_pub['type']
            publisher=type_pub['publisher']
            if file_type=='redif' or file_type=='parse':
                if file_type=='parse': publisher='src'
                print publisher,source,journal_name,
                i=0
                e=0
                for file_name in os.listdir(redif_folder+publisher+'/'+source):
                    if file_name[-4:]=='.rdf' or file_name[-6:]=='.redif':
                        redif_file=publisher+'/'+source+'/'+file_name
                        content = []
                        articles=[]
                        with codecs.open(redif_folder+redif_file,encoding='utf-8',errors='ignore') as file:
                            for line in file:
                                line = line.strip()
                                content.append(line)
                        jsonld_file=redif2jsonld(jsonld_src,journal_name,issn,file_name)
                    i=i+1
                print ', INVALID=',e,'files=',i

38
eee aosoci Accounting, Organizations and Society , INVALID= 5 files= 1
src cnacre Contemporary Accounting Research , INVALID= 1 files= 1
eee jaecon Journal of Accounting and Economics , INVALID= 61 files= 1
bla joares Journal of Accounting Research , INVALID= 0 files= 77
aea aecrev American Economic Review , INVALID= 0 files= 151
ecm emetrp Econometrica , INVALID= 0 files= 29
wly emetrp Econometrica , INVALID= 0 files= 17
ucp jpolec Journal of Political Economy , INVALID= 2 files= 290
oup restud Review of Economic Studies , INVALID= 4 files= 74
oup qjecon The Quarterly Journal of Economics , INVALID= 2 files= 140
cup jfinqa Journal of Financial and Quantitative Analysis , INVALID= 1 files= 52
eee jfinec Journal of Financial Economics , INVALID= 72 files= 1
oup revfin Review of Finance , INVALID= 2 files= 13
oup rfinst Review of Financial Studies , INVALID= 1 files= 4
bla jfinan The Journal of Finance , INVALID= 1692 files= 223
src adscqt Administrative Science Quarterly , INVALID= 2

In [54]:
with open('../OpenScience/subject_journals.json') as data_file:    
    subject_journals = json.load(data_file)
journals=[]
for subject in subject_journals:
    journals=journals+(subject['journals'])
issns=[journal['issn'] for journal in journals if journal['issn'] != '']
    
jsonld_src_folder='jsonld_src/'
jsonld_folder='jsonld/'

for issn in issns:
    with open(jsonld_folder+'doi_'+issn+'.jsonld') as data_file:    
        doi_json = json.load(data_file)
        dois=[art['e:doi'] for art in doi_json]; 
        titles=[art['e:title'] for art in doi_json]; 
    print issn , len(dois),
    new_articles=[]
    for issn_json in os.listdir(jsonld_src_folder):
        if issn_json[0:9]==issn:
            with open(jsonld_src_folder+issn_json) as data_file:    
                src_json = json.load(data_file)
            for src_article in src_json:
                new_article={}
                if 'e:doi' in src_article:
                    src_doi=src_article['e:doi']
                    src_title=src_article['e:title']
                    if src_doi in dois or src_title in titles:
                        if src_doi in dois:
                            doi_article=[art for art in doi_json if art['e:doi']==src_doi][0]
                        if src_title in titles:
                            doi_article=[art for art in doi_json if art['e:title']==src_title][0]
                        for item in ['e:year','e:volume','e:issue','e:keyword','e:abstract']:
                            if item in src_article and item not in doi_article:
                                new_article[item]=src_article[item]
                        new_article['@id']=doi_article['@id']
                    else:
                        if issn in ['0742-1222','2162-9730']:
                            if src_article['e:year']>=1990:
                                for item in src_article: #including issn for jmis
                                    new_article[item]=src_article[item]
                                new_article['@id']=new_article.pop('e:url')
                            
                if len(new_article)>0:
                    new_articles.append(new_article)
            
    jd=unicode(json.dumps(new_articles, ensure_ascii=False))
    jsonld_file=jsonld_folder+'src_'+issn+'.jsonld'
    #if not os.path.exists(jsonld_file):
    with io.open(jsonld_file, 'w', encoding="utf-8") as f:
        f.write(jd)
    print len(new_articles)

0361-3682 1037 852
0823-9150 1236 683
0165-4101 897 693
0021-8456 964 551
0001-4826 1159 0
0002-8282 4016 3506
0012-9682 1289 524
0022-3808 1259 1207
0034-6527 1272 1140
0033-5533 1253 1167
0022-1090 1103 1054
0304-405X 2167 1654
1572-3097 685 547
0893-9454 1656 1094
0022-1082 3049 1452
0001-4273 1960 0
0363-7425 2417 0
0001-8392 1368 653
0149-2063 2086 1649
0022-2380 1481 916
0048-7333 2730 2062
0143-2095 2361 313
0093-5301 1575 921
0732-2399 1234 1213
0022-2429 1247 740
0022-2437 1645 961
0092-0703 1530 0
1047-7047 864 807
0742-1222 931 1184
2162-9730 0 1049
0272-6963 1054 0
0025-1909 4138 3789
1523-4614 631 609
0030-364X 2632 2598
1059-1478 1478 886
1047-7039 1570 1550
0170-8406 2229 2211
0749-5978 1620 1267
